# To calculate the p values an other measurements of statistic signifiants

p-value of kepler energy for the sum of all services

In [10]:
import pandas as pd
from scipy.stats import ttest_ind

# Load CSV
df = pd.read_csv("pods_kepler_joules_all_absolute_joules.csv")


# Identify measurement columns (exclude metadata columns)
measurement_columns = df.columns.difference(['report', 'label', 'time_normalized_rounded'])

df[measurement_columns] = df[measurement_columns].fillna(0)

# Step 1: Calculate total energy for each row (i.e., per time frame)
df['total_energy'] = df[measurement_columns].sum(axis=1)


# Step 2: Define the baseline report
baseline_report = "report_2025-07-07_18-08-43.yaml"
baseline_data = df[df['report'] == baseline_report]['total_energy']

# Step 3: Filter all other reports for comparison
comparison_reports = df[df['report'] != baseline_report]

# Step 4: Compute p-values for each (report, label) vs. baseline
p_values = []

for (report, label), group in comparison_reports.groupby(['report', 'label']):
    variant_data = group['total_energy']
    
    if len(baseline_data) > 1 and len(variant_data) > 1:
        t_stat, p_val = ttest_ind(baseline_data, variant_data, equal_var=False)
    else:
        p_val = None  # Not enough data to compute p-value

    p_values.append({
        'report': report,
        'label': label,
        'p_value': p_val
    })

# Step 5: Create result DataFrame
p_values_df = pd.DataFrame(p_values)

# Optional: Print or save
print(p_values_df)
# p_values_df.to_csv("p_values_vs_baseline.csv", index=False)


                             report              label   p_value
0   report_2025-07-07_18-42-39.yaml                 0%  0.985594
1   report_2025-07-07_19-16-37.yaml                 1s  0.935867
2   report_2025-07-07_19-53-03.yaml                30s  0.982649
3   report_2025-07-07_20-28-01.yaml                 5%  0.978205
4   report_2025-07-07_21-02-07.yaml                10%  0.993811
5   report_2025-07-07_21-36-05.yaml                50%  0.942890
6   report_2025-07-07_22-10-05.yaml   Base Persistence  0.892854
7   report_2025-07-07_22-44-06.yaml     5% Persistence  0.897610
8   report_2025-07-07_23-18-08.yaml    10% Persistence  0.816418
9   report_2025-07-07_23-52-11.yaml    50% Persistence  0.363706
10  report_2025-07-08_00-26-22.yaml     1s Persistence  0.889532
11  report_2025-07-08_01-01-23.yaml    30s Persistence  0.987578
12  report_2025-07-08_01-36-59.yaml  Persistence Istio  0.482730


In [37]:
import pandas as pd
from scipy.stats import ttest_ind

# Load CSV
df = pd.read_csv("pods_kepler_joules_all_absolute_joules.csv")

df.columns = df.columns.str.replace("_system-under-evaluation", "", regex=False)

order = [
    "Baseline",
    "0%",
    "5%",
    "10%",
    "50%",
    "1s",
    "30s",
    "Base Persistence",
    "5% Persistence",
    "10% Persistence",
    "50% Persistence",
    "1s Persistence",
    "30s Persistence",
    "Persistence Istio"
]


# Identify measurement columns (exclude metadata)
measurement_columns = df.columns.difference(['report', 'label', 'time_normalized_rounded'])

# Treat missing energy as 0 (if container not deployed)
df[measurement_columns] = df[measurement_columns].fillna(0)

# Define baseline
baseline_report = "report_2025-07-07_18-08-43.yaml"
baseline_df = df[df['report'] == baseline_report]
comparison_df = df[df['report'] != baseline_report]

# Compute p-values
p_values = []

for (report, label), group_df in comparison_df.groupby(['report', 'label']):
    result_row = {
        'report': report,
        'label': label,
    }

    for column in measurement_columns:
        baseline_values = baseline_df[column]
        variant_values = group_df[column]

        if len(baseline_values) > 1 and len(variant_values) > 1:
            t_stat, p_val = ttest_ind(baseline_values, variant_values, equal_var=False)
            # Mark significant values (p < 0.05) in bold
            if p_val < 0.05:
                formatted_p = f"**{p_val:.4f}** (*)"
            else:
                formatted_p = f"{p_val:.4f}"
        else:
            formatted_p = "N/A"

        result_row[f'p_cpu&mem_{column}'] = formatted_p

    p_values.append(result_row)


# reorder the DataFrame based on the specified order
p_values_df = pd.DataFrame(p_values)

p_values_df['label'] = pd.Categorical(p_values_df['label'], categories=order, ordered=True)
p_values_df = p_values_df.sort_values('label')


print(p_values_df.to_markdown(index=False))


| report                          | label             | p_cpu&mem_accounting   |   p_cpu&mem_ad |   p_cpu&mem_cart | p_cpu&mem_checkout   |   p_cpu&mem_configfile | p_cpu&mem_currency   | p_cpu&mem_elasticsearch   |   p_cpu&mem_email | p_cpu&mem_exporter   | p_cpu&mem_flagd   |   p_cpu&mem_flagd-ui |   p_cpu&mem_fraud-detection |   p_cpu&mem_frontend |   p_cpu&mem_frontend-proxy |   p_cpu&mem_grafana |   p_cpu&mem_image-provider |   p_cpu&mem_init-config |   p_cpu&mem_istio-init | p_cpu&mem_istio-proxy   | p_cpu&mem_jaeger   | p_cpu&mem_jaeger-agent-sidecar   | p_cpu&mem_jaeger-collector   | p_cpu&mem_jaeger-query   | p_cpu&mem_kafka   |   p_cpu&mem_master | p_cpu&mem_opensearch   |   p_cpu&mem_opentelemetry-collector |   p_cpu&mem_payment |   p_cpu&mem_product-catalog |   p_cpu&mem_prometheus-server |   p_cpu&mem_quote |   p_cpu&mem_recommendation |   p_cpu&mem_shipping | p_cpu&mem_valkey-cart   |   p_cpu&mem_wait-for-kafka |   p_cpu&mem_wait-for-valkey-cart |   p_cpu&mem_worker |
|:-